In [48]:
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn
import cv2
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier

import tensorflow as tf
import keras
from keras.utils.np_utils import to_categorical
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from imutils import paths
from keras import metrics
from keras.applications.imagenet_utils import preprocess_input as preprocess_type1

from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

In [39]:
data = []
labels = []

imagePaths = sorted(list(paths.list_images('/home/jailsonpereira/PAIC_CNN/dataset')))
random.seed(42)
random.shuffle(imagePaths)

for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image,(224,224))
    data.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    

data = np.array(data,dtype='float32')/255
labels = np.array(labels)
print(data.shape)

(223, 224, 224, 3)


In [40]:
#le=LabelEncoder()
#labels=le.fit_transform(labels)
#labels=to_categorical(labels,num_classes=2)

X   =  preprocess_type1(data) 

In [41]:
#X_train,X_val,Y_train,Y_val=train_test_split(data,labels,test_size=0.1)

In [42]:
X_net = keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet',pooling='avg').predict(X)

In [52]:
X_net.shape

(223, 2048)

# Abordagem 1

In [46]:
clf = LinearSVC ( ) 
results = cross_val_score(clf,X_net,labels,cv = 5,n_jobs = - 1 ) 


In [47]:
print(results)
print("Overall accuracy: {:.3}".format(np.mean(results) * 100.))

[0.7173913  0.75555556 0.79545455 0.77272727 0.75      ]
Overall accuracy: 75.8


# Abordagem 2

In [70]:
from sklearn.model_selection import GridSearchCV

In [71]:
rate  = [0.01, 0.05]
alpha = [0.5, 2, 3]

neuron_out = 2
neuron_ini = 2048
activation_functions = ['identity', 'logistic', 'tanh', 'relu']
solver = ['lbfgs','adam','sgd']

In [72]:
n = []
for a in alpha:
    n.append(int( a * np.sqrt((neuron_ini*neuron_out))))
print("Quantidade de neurônios nas camadas ocultas a serem testadas respectivamente: ", n)

Quantidade de neurônios nas camadas ocultas a serem testadas respectivamente:  [32, 128, 192]


In [73]:
hidden_layer = [(16,16),(16,14,2),(64,64),(64,64,64)]

In [75]:
parameters = dict([
                ('solver',solver),
                ('hidden_layer_sizes', hidden_layer),
                ('learning_rate_init', rate),
                ('activation', activation_functions)
            ])

In [77]:
clf = GridSearchCV(MLPClassifier(), parameters, iid=True, cv = 3, return_train_score=True)

In [78]:
clf.fit(X_net,labels)

/home/jailsonpereira/anaconda3/envs/paic2/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/jailsonpereira/anaconda3/envs/paic2/lib/python3.6/site-packages/sklearn/preprocessing/label.py:743: RuntimeWarning: invalid value encountered in greater
  y = np.array(y > threshold, dtype=np.int)
/home/jailsonpereira/anaconda3/envs/paic2/lib/python3.6/site-packages/sklearn/preprocessing/label.py:743: RuntimeWarning: invalid value encountered in greater
  y = np.array(y > threshold, dtype=np.int)
/home/jailsonpereira/anaconda3/envs/paic2/lib/python3.6/site-packages/sklearn/preprocessing/label.py:743: RuntimeWarning: invalid value encountered in greater
  y = np.array(y > threshold, dtype=np.int)
/home/jailsonpereira/anaconda3/envs/paic2/lib/python3.6/site-packages/sklearn/preprocessing/label.py:743: Runti

/home/jailsonpereira/anaconda3/envs/paic2/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/jailsonpereira/anaconda3/envs/paic2/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/jailsonpereira/anaconda3/envs/paic2/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/jailsonpereira/anaconda3/envs/paic2/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimi

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=None,
       param_grid={'solver': ['lbfgs', 'adam', 'sgd'], 'hidden_layer_sizes': [(16, 16), (16, 14, 2), (64, 64), (64, 64, 64)], 'learning_rate_init': [0.01, 0.05], 'activation': ['identity', 'logistic', 'tanh', 'relu']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [80]:
import pandas as pd
resultado = pd.DataFrame.from_dict(clf.cv_results_)
resultado

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_hidden_layer_sizes,param_learning_rate_init,param_solver,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.841032,0.041939,0.001417,0.000015,identity,"(16, 16)",0.01,lbfgs,"{'activation': 'identity', 'hidden_layer_sizes...",0.680000,0.716216,0.648649,0.681614,0.027546,69,0.905405,0.906040,0.912752,0.908066,0.003324
1,0.133861,0.115111,0.001406,0.000012,identity,"(16, 16)",0.01,adam,"{'activation': 'identity', 'hidden_layer_sizes...",0.693333,0.810811,0.675676,0.726457,0.059883,4,0.743243,0.691275,0.751678,0.728732,0.026709
2,0.221121,0.131595,0.001374,0.000021,identity,"(16, 16)",0.01,sgd,"{'activation': 'identity', 'hidden_layer_sizes...",0.000000,0.810811,0.675676,0.493274,0.355434,86,0.000000,0.684564,0.751678,0.478747,0.339632
3,0.803209,0.007865,0.001420,0.000016,identity,"(16, 16)",0.05,lbfgs,"{'activation': 'identity', 'hidden_layer_sizes...",0.640000,0.702703,0.621622,0.654709,0.034647,79,0.932432,0.912752,0.939597,0.928260,0.011350
4,0.027936,0.000150,0.001425,0.000009,identity,"(16, 16)",0.05,adam,"{'activation': 'identity', 'hidden_layer_sizes...",0.253333,0.189189,0.675676,0.372197,0.215471,89,0.378378,0.315436,0.751678,0.481831,0.192533
5,0.282148,0.181594,0.001510,0.000025,identity,"(16, 16)",0.05,sgd,"{'activation': 'identity', 'hidden_layer_sizes...",0.000000,0.000000,0.000000,0.000000,0.000000,93,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.850900,0.029846,0.001436,0.000038,identity,"(16, 14, 2)",0.01,lbfgs,"{'activation': 'identity', 'hidden_layer_sizes...",0.666667,0.716216,0.648649,0.677130,0.028513,71,0.945946,0.906040,0.919463,0.923816,0.016580
7,0.190762,0.228613,0.001474,0.000008,identity,"(16, 14, 2)",0.01,adam,"{'activation': 'identity', 'hidden_layer_sizes...",0.000000,0.810811,0.662162,0.488789,0.353183,87,0.000000,0.684564,0.758389,0.480984,0.341440
8,0.069846,0.011669,0.001412,0.000004,identity,"(16, 14, 2)",0.01,sgd,"{'activation': 'identity', 'hidden_layer_sizes...",0.693333,0.810811,0.675676,0.726457,0.059883,4,0.743243,0.684564,0.751678,0.726495,0.029849
9,0.841238,0.034213,0.001766,0.000452,identity,"(16, 14, 2)",0.05,lbfgs,"{'activation': 'identity', 'hidden_layer_sizes...",0.666667,0.689189,0.675676,0.677130,0.009262,71,0.912162,0.879195,0.946309,0.912555,0.027401


In [82]:
print("Melhor média de acurácia entre os folds = " + str(max(resultado['mean_train_score'])))

Melhor média de acurácia entre os folds = 0.9283058226011246


In [83]:
resultado.loc[resultado['mean_train_score']==max(resultado['mean_train_score'])]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_hidden_layer_sizes,param_learning_rate_init,param_solver,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
18,2.847417,0.076621,0.001888,0.000273,identity,"(64, 64, 64)",0.01,lbfgs,"{'activation': 'identity', 'hidden_layer_sizes...",0.613333,0.689189,0.662162,0.654709,0.031444,79,0.952703,0.865772,0.966443,0.928306,0.044573
